<a href="https://colab.research.google.com/github/kitomicms/python_chess_/blob/master/python_chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# python chess
# class Game():
#   def __init__(self):
#     board = Board()
#     self.board = board
#     self.turn = 'W'

class Board():
  def __init__(self):
        files = ['A','B','C','D','E','F','G','H']
        ranks = [1,2,3,4,5,6,7,8]
        board = []
        counter = 1

        # create all the squares
        for rank in reversed(ranks):
          for file in files:
            board.append(Square(rank,file,counter))
            counter += 1

        # add coordinates
        row_counter = 1
        for count,square in enumerate(board):
          
          if (count+1) % 8 == 0:
            x = 8
            y = row_counter
            row_counter +=1
          else:
            x = (count+1) % 8
            y = row_counter
          square.coordinate = (x,y)
            

        # add pawns
        for square in board:
          if square.rank == 2:
            square.addPiece("P","W")
          elif square.rank == 7:
            square.addPiece("P","B")
        # add other pieces
        abc = ['R','N','B','Q','K','B','N','R']
        counter = 0
        for square in board:
          if square.rank == 1:
            square.addPiece(abc[counter],"W")
            counter += 1
        counter = 0
        for square in board:
          if square.rank == 8:
            square.addPiece(abc[counter],"B")
            counter += 1
        self.board = board
        
        # set initial turn
        self.turn = 'W'

  def current_turn(self):
    return self.turn

  def display_matrix(self):
    display_str = ''
    for count,square in enumerate(self.board):
      if (count+1) % 8 == 0:
          display_str = f"{display_str}{square.coordinate} "
          print(display_str)
          display_str = ''
      else:
        display_str = f"{display_str}{square.coordinate} "

  def display_board(self):
      display_str = ''
      for count,square in enumerate(self.board):
        if (count+1) % 8 == 0:
          display_str = f"{display_str}{square.piece_color}{square.piece_name} "
          print(display_str)
          display_str = ''
        else:
          display_str = f"{display_str}{square.piece_color}{square.piece_name} "
      #print(display_str)

  def square_info(self,rank,file):
    # what is on the square?
    
    for square in self.board:
      if square.rank == rank and square.file == file:
        return [square,square.piece_color,square.piece_name,square.coordinate]


  def makeMove(self,square1,square2):
    sq1_ = self.square_info(int(square1[1]),square1[0])[0]
    sq1_color = self.square_info(int(square1[1]),square1[0])[1]
    sq1_pieceName = self.square_info(int(square1[1]),square1[0])[2]
    sq2_ = self.square_info(int(square2[1]),square2[0])[0]
    sq2_color = self.square_info(int(square2[1]),square2[0])[1]
    sq2_pieceName = self.square_info(int(square2[1]),square2[0])[2]

    if self.turn != sq1_color:
      print("Not your turn")
      return 0

    if sq1_.isOccupied != False:
      if sq2_.isOccupied == False:
        sq1_.reset()
        sq2_.reset()
        sq2_.addPiece(sq1_pieceName,sq1_color)
        self.turn = 'W' if self.turn == 'B' else 'B'

      elif sq2_.isOccupied and sq1_color != sq2_color:
        sq1_.reset()
        sq2_.reset()
        sq2_.addPiece(sq1_pieceName,sq1_color)
        self.turn = 'W' if self.turn == 'B' else 'B'
      elif sq1_color == sq2_color:
        print("both square has same color piece")
      else:
        print("something else went wrong")
    else:
      print("Square 1 is Empty")
    # 1. MT
    # 2. Occupied,MT (King exception, path checked by opponent)
    # 3. Occupied,Occupied
    # 3.1 W,W
    # 3.2 W,B
    
  def validMoves(square1):
    pass

# row = rank, col = file
class Square():
  def __init__(self, rank,file,coordinate,piece=None):
        self.rank = rank
        self.file = file
        self.isOccupied = False
        self.piece = None
        self.piece_name = 'X'
        self.piece_color = 'X'
        self.coordinate = coordinate

  def addPiece(self,name,color):
    if name == 'P':
      self.piece = Pawn(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'K':
      self.piece = King(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'Q':
      self.piece = Queen(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'R':
      self.piece = Rook(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'N':
      self.piece = Knight(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    elif name == 'B':
      self.piece = Bishop(name,color)
      self.piece_name = name
      self.piece_color = color
      self.isOccupied = True
    else:
      print("invalid piece name entered")
      

  def reset(self):
    self.isOccupied = False
    self.piece = None
    self.piece_name = 'X'
    self.piece_color = 'X'

class Piece():
  def __init__(self,name,color):
    self.name = name
    self.color = color


class King(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = [1,-1,8,-8,7,9,-7,-9]

class Queen(Piece):
  # moves rule
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = [1,2,3,4,-1,-2,-3,-4,8,16,24,32,-8,-16,-24,-32,7,14,21,28,-7,-14,-21,-28,9,18,27,36,-9,-18,-27,-36]

class Rook(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = [1,2,3,4,-1,-2,-3,-4,8,16,24,32,-8,-16,-24,-32]
class Pawn(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    # depends on which color
    self.validMovesIndex = [1,-1]
class Knight(Piece):
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = [6,10,15,17,-6,-10,-15,-17]

class Bishop(Piece):
  
  def __init__(self, name,color):
    self.name = name
    self.color = color
    self.validMovesIndex = [7,14,21,28,-7,-14,-21,-28,9,18,27,36,-9,-18,-27,-36]




In [27]:
board = Board()
board.current_turn()
board.makeMove("B2","B4")
board.display_board()
#board.makeMove("B2","B4")
#board.display_board()

BR BN BB BQ BK BB BN BR 
BP BP BP BP BP BP BP BP 
XX XX XX XX XX XX XX XX 
XX XX XX XX XX XX XX XX 
XX WP XX XX XX XX XX XX 
XX XX XX XX XX XX XX XX 
WP XX WP WP WP WP WP WP 
WR WN WB WQ WK WB WN WR 


In [23]:
board.makeMove("B4","D4")
board.display_board()

Not your turn
BR BN BB BQ BK BB BN BR 
BP BP BP BP BP BP BP BP 
XX XX XX XX XX XX XX XX 
XX XX XX XX XX XX XX XX 
XX WP XX XX XX XX XX XX 
XX XX XX XX XX XX XX XX 
WP XX WP WP WP WP WP WP 
WR WN WB WQ WK WB WN WR 


In [28]:

board.display_matrix()

(1, 1) (2, 1) (3, 1) (4, 1) (5, 1) (6, 1) (7, 1) (8, 1) 
(1, 2) (2, 2) (3, 2) (4, 2) (5, 2) (6, 2) (7, 2) (8, 2) 
(1, 3) (2, 3) (3, 3) (4, 3) (5, 3) (6, 3) (7, 3) (8, 3) 
(1, 4) (2, 4) (3, 4) (4, 4) (5, 4) (6, 4) (7, 4) (8, 4) 
(1, 5) (2, 5) (3, 5) (4, 5) (5, 5) (6, 5) (7, 5) (8, 5) 
(1, 6) (2, 6) (3, 6) (4, 6) (5, 6) (6, 6) (7, 6) (8, 6) 
(1, 7) (2, 7) (3, 7) (4, 7) (5, 7) (6, 7) (7, 7) (8, 7) 
(1, 8) (2, 8) (3, 8) (4, 8) (5, 8) (6, 8) (7, 8) (8, 8) 


In [ ]:
king.color

'W'

In [15]:
board.square_info(1,"B")

[<__main__.Square at 0x7f1e026922d0>, 'W', 'N', (8, 8)]